In [3]:
import os
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
from scipy.stats import entropy
from sklearn.preprocessing import normalize
import random

def add_noise(image, noise_factor):
    """添加高斯噪声"""
    img_array = np.array(image)
    noise = np.random.normal(0, noise_factor * 255, img_array.shape)
    noisy_img_array = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_img_array)

def change_brightness(image, factor):
    """改变图像亮度"""
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(factor)

def change_contrast(image, factor):
    """改变图像对比度"""
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(factor)

def apply_filter(image, filter_type):
    """应用滤镜"""
    if filter_type == "blur":
        return image.filter(ImageFilter.BLUR)
    elif filter_type == "sharpen":
        return image.filter(ImageFilter.SHARPEN)
    elif filter_type == "edge_enhance":
        return image.filter(ImageFilter.EDGE_ENHANCE)
    else:
        return image

def random_transform(image, intensity):
    """随机应用一种或多种变换"""
    transforms = [
        lambda img: add_noise(img, intensity * 0.1),
        lambda img: change_brightness(img, 1 + (intensity - 0.5)),
        lambda img: change_contrast(img, 1 + (intensity - 0.5)),
        lambda img: apply_filter(img, random.choice(["blur", "sharpen", "edge_enhance"]))
    ]
    
    num_transforms = random.randint(1, len(transforms))
    selected_transforms = random.sample(transforms, num_transforms)
    
    for transform in selected_transforms:
        image = transform(image)
    
    return image

def process_images_in_directory(directory_path, intensity):
    """处理指定目录下的所有图片并覆盖原图"""
    for filename in os.listdir(directory_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            file_path = os.path.join(directory_path, filename)
            with Image.open(file_path) as img:
                transformed_img = random_transform(img, intensity)
                transformed_img.save(file_path)

In [7]:
# 处理指定目录下的图片
directory_path = "/data/office_home_adjusted/art/train/Alarm_Clock"
process_images_in_directory(directory_path, intensity=0.5)

FileNotFoundError: [Errno 2] No such file or directory: '/data/office_home_adjusted/art/train/Alarm_Clock'

In [ ]:
def image_to_histogram(image_path, bins=256):
    """将图像转换为颜色直方图"""
    img = Image.open(image_path).convert('RGB')
    img_array = np.array(img)
    hist = np.histogramdd(img_array.reshape(-1, 3), bins=bins, range=[(0, 256)] * 3)[0]
    return hist.flatten() / hist.sum()

def kl_divergence(p, q):
    """计算KL散度"""
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def compute_average_kl_divergence(source_dir, target_dir):
    """计算源域和目标域之间的平均KL散度"""
    source_hists = []
    target_hists = []

    # 收集源域图像的直方图
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                hist = image_to_histogram(os.path.join(root, file))
                source_hists.append(hist)

    # 收集目标域图像的直方图
    for root, _, files in os.walk(target_dir):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                hist = image_to_histogram(os.path.join(root, file))
                target_hists.append(hist)

    # 计算平均KL散度
    kl_divergences = []
    for source_hist in source_hists:
        for target_hist in target_hists:
            kl_div = kl_divergence(source_hist + 1e-10, target_hist + 1e-10)
            kl_divergences.append(kl_div)

    return np.mean(kl_divergences)

# 计算KL散度
source_dir = "data/office_home_adjusted/clipart"
target_dir = "data/office_home_adjusted/art"
avg_kl_divergence = compute_average_kl_divergence(source_dir, target_dir)
print(f"Average KL Divergence between source and target: {avg_kl_divergence}")